In [1]:
import numpy as np

In [9]:
def production(actions):
    mils = 32
    docks = 6
    civs = 45
    total = mils + docks + civs
    consumergoods_percentage = 0.3
    consumergoods = np.ceil(total * consumergoods_percentage)
    available_civs = np.maximum(int(civs - consumergoods), 0)
    civ_output = 5 
    mil_output = 5
    state = 0
    state_size = 100 #how many days per state
    ic_produced = 0
    ic_total = 0    
    civ_purchase_pool = 0 
    mil_purchase_pool = 0
    
    for i in range(len(actions)):
        if actions[i] == 0: #build mils     
            ic_produced = mil_output * mils * 100
            mil_purchase_pool += available_civs * civ_output * 100
            mils += np.floor_divide(mil_purchase_pool, 7200) 
            total += np.floor_divide(mil_purchase_pool, 7200) 
            consumergoods = np.ceil(total * 0.3)
            mil_purchase_pool = np.mod(mil_purchase_pool, 7200)
            state += 1
            ic_total += ic_produced
        if actions[i] == 1:
            ic_produced = mil_output * mils * 100
            civ_purchase_pool += available_civs * civ_output * 100
            civs += np.floor_divide(civ_purchase_pool, 7200) 
            total += np.floor_divide(civ_purchase_pool, 7200) 
            consumergoods = np.ceil(total * 0.3)
            civ_purchase_pool = np.mod(civ_purchase_pool, 7200)
            state += 1
            ic_total += ic_produced
    
    return ic_total

In [10]:
production([0,0,0,0])

67500

In [239]:
def selection(pop, scores, k=3):
 # first random selection
    selection_ix = np.random.randint(len(pop))
    for ix in np.random.randint(0, len(pop), k-1):
        if scores[ix] > scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    if np.random.rand() < r_cross:
        pt = np.random.randint(1, len(p1)-1)
        #print(pt)
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if np.random.rand() < r_mut:
            bitstring[i] = 1 - bitstring[i]
    return bitstring

def genetic_algorithm(n_bits, n_pop, n_iter, r_cross):
    pop = [np.random.randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
    best, best_eval = 0, production(pop[0])
    
    for gen in range(n_iter):
        scores = [production(c) for c in pop]
        #print(pop, scores)
        for i in range(n_pop):
            if scores[i] > best_eval:
                best, best_eval = pop[i], scores[i]
                print("Generation " + str(gen) + ": new best solution found!")
        
        selected = [selection(pop, scores) for _ in range(n_pop)]
        children = []
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            c = crossover(p1, p2, r_cross)
            for child in c:
                mutated = mutation(child, 1 / n_bits)
                children.append(mutated)
        #print(np.mean(sum([production(c) for c in pop])))
        #print(np.mean(sum([production(c) for c in children])))
        print(sum([sum(strat) for strat in pop])) #lower is better in this simple case
        pop = children
        
        
    return [best, best_eval]
    #return sum([sum(strat) for strat in pop]), sum([sum(strat) for strat in children])

In [240]:
genetic_algorithm(10,10, 5, 0.9)

Generation 0: new best solution found!
Generation 0: new best solution found!
49
44
37
42
Generation 4: new best solution found!
40


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 189000]

In [17]:
crossover([0, 0, 0, 0, 0], [1, 1, 1, 1, 1], 0.9)

2


[[0, 0, 1, 1, 1], [1, 1, 0, 0, 0]]

In [18]:
pop = [np.random.randint(0, 2, 4).tolist() for _ in range(10)]
scores = [production(c) for c in pop]

In [32]:
mutation([0, 0, 0, 0, 0], 1/6)

[0, 0, 0, 0, 0]

In [242]:
production([0,0,0,0,0,0,0,0,0,0])

189000

In [244]:
production([1,1,1,0,0,0,0,0,0,0])

173000